In [ ]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[incanter "1.5.7"])
(use '(incanter core stats charts io datasets ))

In [ ]:
(require '[cemerick.pomegranate :refer [add-dependencies]])
(add-dependencies :coordinates '[[org.clojure/data.csv "0.1.2"]])
(require '[clojure.data.csv :as csv])

In [ ]:
(def games (drop 1 (with-open [in-file (clojure.java.io/reader "games.csv")]
    (doall
         (csv/read-csv in-file)
    )
)))

(defrecord Game [game west_sales global_sales])

(defn toInt [n] (Integer. n))
(defn toDouble [n] (Double. n))
(defn square [n] (* n n))
(defn avg [items] (/ (reduce + items) (count items)))
(defn error [aprox real] (* (/ (abs (- aprox real)) real) 100))

(defn gameToRecord [g] 
    (Game. 
         (nth g 0)
         (if (and (empty? (nth g 5)) (empty? (nth g 6))) 0 (+ (toDouble (nth g 5)) (toDouble (nth g 6))))
         (toDouble (if (empty? (nth g 9)) "0" (nth g 9))) 
    )
)

(def data (map #(gameToRecord %) games))

(ns-unmap (find-ns 'user) 'games)

(def shuffledData (shuffle data))
(def trainingData (take (int (* (count data) 0.8)) shuffledData))
(def testData (take-last (int (* (count data) 0.2)) shuffledData))

(ns-unmap (find-ns 'user) 'data)
(ns-unmap (find-ns 'user) 'shuffledData)

In [ ]:
(def ventasWest (map #(:west_sales %) trainingData))
(def ventasGlobal (map #(:global_sales %) trainingData))

In [ ]:
(-> (scatter-plot ventasWest
                 ventasGlobal
                 :x-label "Ventas Occidente" :y-label "Ventas Mundiales")
    (.createBufferedImage 600 400)
)

In [ ]:
(def avgWest (avg ventasWest))
(def avgGlobal (avg ventasGlobal))

(def m 
    (/ 
        (reduce + (map #(* (- (:west_sales %) avgWest) (- (:global_sales %) avgGlobal)) trainingData))
        (reduce + (map #(square (- (:west_sales %) avgGlobal)) trainingData))
    )
)
(def c (- avgGlobal (* avgWest m)))

(defn eq [x] (+ (* x m) c) )


In [ ]:
(-> (scatter-plot ventasWest
                  ventasGlobal
                  :x-label "Ventas Occidente" :y-label "Ventas Mundiales")
    (add-function eq 0 (+ (reduce max ventasWest) 1))
    (.createBufferedImage 600 400)
)

In [ ]:
;;Promedio de porcentaje de error en datos de prueba
(avg (map #(error (eq (:west_sales %)) (:global_sales %)) testData))